In [1]:
import warnings
warnings.filterwarnings("ignore")
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
from fastai.vision import *

In [3]:
torch.multiprocessing.set_sharing_strategy('file_system')

In [4]:
PATH = Path("/home/ubuntu/data/kaggle")

In [5]:
[print(i) for i in PATH.iterdir()]

/home/ubuntu/data/kaggle/test.csv.zip
/home/ubuntu/data/kaggle/test_images.zip
/home/ubuntu/data/kaggle/sample_submission.csv.zip
/home/ubuntu/data/kaggle/train.csv
/home/ubuntu/data/kaggle/images
/home/ubuntu/data/kaggle/models
/home/ubuntu/data/kaggle/test.csv
/home/ubuntu/data/kaggle/test


[None, None, None, None, None, None, None, None]

In [6]:
df = pd.read_csv(PATH/"train.csv")

In [7]:
df.head()

,category_id,date_captured,file_name,frame_num,id,location,rights_holder,seq_id,seq_num_frames,width,height
0,19,2011-05-13 23:43:18,5998cfa4-23d2-11e8-a6a3-ec086b02610b.jpg,1,5998cfa4-23d2-11e8-a6a3-ec086b02610b,33,Justin Brown,6f084ccc-5567-11e8-bc84-dca9047ef277,3,1024,747
1,19,2012-03-17 03:48:44,588a679f-23d2-11e8-a6a3-ec086b02610b.jpg,2,588a679f-23d2-11e8-a6a3-ec086b02610b,115,Justin Brown,6f12067d-5567-11e8-b3c0-dca9047ef277,3,1024,747
2,0,2014-05-11 11:56:46,59279ce3-23d2-11e8-a6a3-ec086b02610b.jpg,1,59279ce3-23d2-11e8-a6a3-ec086b02610b,96,Erin Boydston,6faa92d1-5567-11e8-b1ae-dca9047ef277,1,1024,747
3,0,2013-10-06 02:00:00,5a2af4ab-23d2-11e8-a6a3-ec086b02610b.jpg,1,5a2af4ab-23d2-11e8-a6a3-ec086b02610b,57,Erin Boydston,6f7d4702-5567-11e8-9e03-dca9047ef277,1,1024,747
4,0,2011-07-12 13:11:16,599fbd89-23d2-11e8-a6a3-ec086b02610b.jpg,3,599fbd89-23d2-11e8-a6a3-ec086b02610b,46,Justin Brown,6f1728a1-5567-11e8-9be7-dca9047ef277,3,1024,747


In [8]:
df_animals = df[df.category_id != 0] #removing null class from dataset

In [9]:
classes = df_animals["category_id"].unique()

In [10]:
classes

array([19,  3,  8,  4, 13,  1, 11, 16, 17, 14, 18, 10, 22])

In [11]:
classes_map = {classes[i]:i for i in range(len(classes))}

In [12]:
classes_map

{19: 0,
 3: 1,
 8: 2,
 4: 3,
 13: 4,
 1: 5,
 11: 6,
 16: 7,
 17: 8,
 14: 9,
 18: 10,
 10: 11,
 22: 12}

In [13]:
new_labels = [ classes_map[i] for i in df_animals["category_id"]]

In [14]:
new_labels[0]

0

In [15]:
df_animals["new_class"] = new_labels

In [16]:
train_df = df_animals[["file_name","new_class"]]

In [17]:
train_df.columns = ['name', 'label']

In [18]:
train_df.head()

,name,label
0,5998cfa4-23d2-11e8-a6a3-ec086b02610b.jpg,0
1,588a679f-23d2-11e8-a6a3-ec086b02610b.jpg,0
5,593d68d7-23d2-11e8-a6a3-ec086b02610b.jpg,1
6,59fae563-23d2-11e8-a6a3-ec086b02610b.jpg,2
12,595f7b9c-23d2-11e8-a6a3-ec086b02610b.jpg,3


In [23]:
data = ImageDataBunch.from_df(PATH, train_df, "images", size=224, bs=128)

In [20]:
#data.show_batch(rows=3, figsize=(5,5))

In [25]:
from fastai.metrics import error_rate, accuracy
from fastai.vision import models 
from fastai.vision import learner

In [26]:
learn = learner.create_cnn(data, models.resnet34, metrics=[error_rate,accuracy])

In [27]:
learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,1.056824,0.866527,0.296036,0.703964,3:53:11
1,0.807945,0.675602,0.230028,0.769972,3:52:46
2,0.645088,0.548816,0.189081,0.810919,3:52:12
3,0.560373,0.518918,0.175278,0.824722,3:52:48


In [28]:
learn.save("model3")

In [29]:
learn.export()

In [30]:
df_test = pd.read_csv(PATH/"test.csv")

In [31]:
df.head()

,category_id,date_captured,file_name,frame_num,id,location,rights_holder,seq_id,seq_num_frames,width,height
0,19,2011-05-13 23:43:18,5998cfa4-23d2-11e8-a6a3-ec086b02610b.jpg,1,5998cfa4-23d2-11e8-a6a3-ec086b02610b,33,Justin Brown,6f084ccc-5567-11e8-bc84-dca9047ef277,3,1024,747
1,19,2012-03-17 03:48:44,588a679f-23d2-11e8-a6a3-ec086b02610b.jpg,2,588a679f-23d2-11e8-a6a3-ec086b02610b,115,Justin Brown,6f12067d-5567-11e8-b3c0-dca9047ef277,3,1024,747
2,0,2014-05-11 11:56:46,59279ce3-23d2-11e8-a6a3-ec086b02610b.jpg,1,59279ce3-23d2-11e8-a6a3-ec086b02610b,96,Erin Boydston,6faa92d1-5567-11e8-b1ae-dca9047ef277,1,1024,747
3,0,2013-10-06 02:00:00,5a2af4ab-23d2-11e8-a6a3-ec086b02610b.jpg,1,5a2af4ab-23d2-11e8-a6a3-ec086b02610b,57,Erin Boydston,6f7d4702-5567-11e8-9e03-dca9047ef277,1,1024,747
4,0,2011-07-12 13:11:16,599fbd89-23d2-11e8-a6a3-ec086b02610b.jpg,3,599fbd89-23d2-11e8-a6a3-ec086b02610b,46,Justin Brown,6f1728a1-5567-11e8-9be7-dca9047ef277,3,1024,747


In [32]:
from fastai.vision import ImageImageList

In [33]:
test_data = ImageImageList.from_df(df_test, PATH/'test', cols="file_name")

In [34]:
learn.data.add_test(test_data)

In [35]:
preds, y = learn.get_preds(ds_type=DatasetType.Test)

In [39]:
np.save(PATH/"preds-baseline", preds)

In [ ]:
"model/model1.pth"

In [41]:
! ls $PATH

export.pkl  preds-baseline.npy	       test.csv		train.csv
images	    sample_submission.csv.zip  test.csv.zip
models	    test		       test_images.zip


In [61]:
learn = load_learner(PATH/"model/model1.pth",test=test_data)

FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/data/kaggle/model/model1.pth/export.pkl'